In [56]:
import sys
import torch
import numpy as np

In [57]:
sys.path.append('/envs')
sys.path.append('/models')
import envs.viper as vpr
import models.models as models
import models.dqn as dqn

In [58]:
import importlib
importlib.reload(vpr); importlib.reload(models); importlib.reload(dqn)

<module 'models.dqn' from 'h:\\Coding Projects\\Python Projects\\Deep-XIV-Rotation\\models\\dqn.py'>

In [59]:
viper_env = vpr.Viper(_sks = 798)

b123


In [60]:
viper_env.get_max_actions()

11

In [61]:
viper_env.get_state_shape()

8

In [62]:
state = viper_env.state()
state[None, :]

tensor([[ 0.0000,  2.4500, 10.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [63]:
state.unsqueeze(0)

tensor([[ 0.0000,  2.4500, 10.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [64]:
agent = dqn.DQN(viper_env)

Layer (type:depth-idx)                   Output Shape              Param #
DenseNet                                 [8, 11]                   --
├─Linear: 1-1                            [8, 2048]                 18,432
├─Linear: 1-2                            [8, 11]                   22,539
Total params: 40,971
Trainable params: 40,971
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.33
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.16
Estimated Total Size (MB): 0.30


In [65]:
torch.device('cpu')

device(type='cpu')

In [66]:
inputs = torch.rand(5, viper_env.get_state_shape())
print(inputs)
preds = agent.get_action(inputs)
print(preds)

tensor([[0.8517, 0.3178, 0.2764, 0.2242, 0.6636, 0.6973, 0.8767, 0.0875],
        [0.3155, 0.1787, 0.3785, 0.2640, 0.5134, 0.3651, 0.8847, 0.3411],
        [0.6504, 0.3710, 0.5037, 0.1070, 0.2346, 0.0189, 0.7085, 0.8012],
        [0.0986, 0.6174, 0.2867, 0.7167, 0.4976, 0.7942, 0.2961, 0.9869],
        [0.3331, 0.6300, 0.3605, 0.3403, 0.8028, 0.8278, 0.5676, 0.8231]])
tensor([9, 9, 9, 3, 9], dtype=torch.int32)


In [67]:
print(viper_env.step(1))
viper_env.state()

Taking action steel_fangs
(200.0, 196.42)


tensor([ 0.7000,  2.4500,  0.7000,  1.0000, 59.3000,  0.0000,  0.0000,  0.0000])

In [68]:
for i in range(25):
    print(viper_env.step(0))
    print(viper_env.state())

Taking action nothing
(-1.0, 0.0)
tensor([ 0.8000,  2.4500,  0.8000,  1.0000, 59.2000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 0.9000,  2.4500,  0.9000,  1.0000, 59.1000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.0000,  2.4500,  1.0000,  1.0000, 59.0000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.1000,  2.4500,  1.1000,  1.0000, 58.9000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.2000,  2.4500,  1.2000,  1.0000, 58.8000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.3000,  2.4500,  1.3000,  1.0000, 58.7000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.4000,  2.4500,  1.4000,  1.0000, 58.6000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.5000,  2.4500,  1.5000,  1.0000, 58.5000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.6000,  2.4500,  1.6000,  1.0000, 58

In [69]:
print(viper_env.step(3))
viper_env.state()

Taking action hunters_sting
(330.0, 343.03)


tensor([ 3.9000,  2.4500,  0.7000,  2.0000, 56.1000,  0.0000, 39.3000,  0.0000])

In [70]:
viper_env.compute_gcd(2.5, 10, 0)

2.5